# Open-EO: Simple Compositing
This notebook explains how to use the OpenEO client library to create basic image composites.

Let's get started with importing the module.

In [1]:
!pip install --user openeo==0.8.2

import openeo
import logging

Looking in indexes: https://artifactory.vgt.vito.be/api/pypi/python-packages/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# logging.basicConfig(level=logging.DEBUG)

To connect with an OpenEO backend, we create a session. Each OpenEO backend has a different endpoint, metadata and credentials. A session object contains this information, and is the starting point for subsequent calls. 

In [3]:
session = openeo.connect("http://openeo.vgt.vito.be").authenticate_oidc("egi")

Authenticated using refresh token.


## Loading a collection

Our first use case is to create a composite image by taking the maximum pixel value over a timeseries of images.
To do this, we first need to select input data. Each OpenEO endpoint exposes it's own list of image collections.

In [4]:
session.list_collections()

[{'description': 'Sentinel 1 GRD Sigma0, mosaic of all ascending relative orbits. Backscatter values are provided as natural number, not in decibel. Provided by Terrascope.\n\n Use [this viewer](https://viewer.terrascope.be/?language=en&bbox=2.8531494643539195,50.29627033037724,6.36877446435392,51.454777120718546&overlay=true&bgLayer=Satellite&date=2020-10-15&layer=CGS_S1_GRD_SIGMA0) to explore the data.',
  'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
   'temporal': {'interval': [['2015-07-06T00:00:00Z', None]]}},
  'id': 'S1_GRD_SIGMA0_ASCENDING',
  'keywords': ['VITO',
   'TERRASCOPE',
   'COPERNICUS',
   'ESA',
   'Orthoimagery',
   'SENTINEL-1',
   'C-SAR',
   'Level-3',
   'Radar',
   'Radar Backscatter'],
  'license': 'free',
  'links': [{'href': 'https://docs.terrascope.be/#/DataProducts/Sentinel-1/ProductsOverview',
    'rel': 'alternate',
    'title': 'Product web page.'},
   {'href': 'https://services.terrascope.be/wmts/v2',
    'rel': 'wmts',
    'wmts:layer': 'CGS_S1_GRD_SIGMA0'}],
  'providers': [{'name': 'Terrascope/VITO'}],
  'stac_extensions': ['datacube'],
  'stac_version': '0.9.0',
  'title': 'Sentinel 1 GRD Sigma0 product, VH, VV and angle.'},
 {'description': 'Sentinel 1 GRD Sigma0, mosaic of all descending relative orbits. Backscatter values are provided as natural number, not in decibel. Provided by Terrascope.\n\n Use [this viewer](https://viewer.terrascope.be/?language=en&bbox=2.8531494643539195,50.29627033037724,6.36877446435392,51.454777120718546&overlay=true&bgLayer=Satellite&date=2020-10-15&layer=CGS_S1_GRD_SIGMA0) to explore the data.',
  'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
   'temporal': {'interval': [['2015-07-06T00:00:00Z', None]]}},
  'id': 'S1_GRD_SIGMA0_DESCENDING',
  'keywords': ['VITO',
   'TERRASCOPE',
   'COPERNICUS',
   'ESA',
   'Orthoimagery',
   'SENTINEL-1',
   'C-SAR',
   'Level-3',
   'Radar',
   'Radar Backscatter'],
  'license': 'free',
  'links': [{'href': 'https://docs.terrascope.be/#/DataProducts/Sentinel-1/ProductsOverview',
    'rel': 'alternate',
    'title': 'Product web page.'},
   {'href': 'https://services.terrascope.be/wmts/v2',
    'rel': 'wmts',
    'wmts:layer': 'CGS_S1_GRD_SIGMA0'}],
  'providers': [{'name': 'Terrascope/VITO'}],
  'stac_extensions': ['datacube'],
  'stac_version': '0.9.0',
  'title': 'Sentinel 1 GRD Sigma0 product, VH, VV and angle.'},
 {'description': 'FAPAR Level-3 product at 10m resolution. Covers selected areas globally. This FAPAR is derived directly from ESA L2A products. The FAPAR quantifies the fraction of the solar radiation absorbed by live leaves for the photosynthesis activity. Then, it refers only to the green and alive elements of the canopy. The FAPAR depends on the canopy structure, vegetation element optical properties, atmospheric conditions, and angular configuration. \n\nFAPAR is recognized as an Essential Climate Variable (ECV) by the Global Climate Observing System (GCOS).',
  'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
   'temporal': {'interval': [['2015-07-06T00:00:00Z', None]]}},
  'id': 'TERRASCOPE_S2_FAPAR_V2',
  'keywords': ['VITO',
   'TERRASCOPE',
   'COPERNICUS',
   'ESA',
   'Orthoimagery',
   'SENTINEL-2',
   'MSI',
   'Level-3',
   'Radiometry',
   'Plant Resource',
   'FAPAR'],
  'license': 'proprietary',
  'links': [{'href': 'https://sentinel.esa.int/documents/247904/690755/Sentinel_Data_Legal_Notice',
    'rel': 'license'},
   {'href': 'https://services.terrascope.be/catalogue/description?collection=urn:eop:VITO:TERRASCOPE_S2_FAPAR_V2',
    'rel': 'alternate'},
   {'href': 'https://viewer.terrascope.be/?language=nl&bbox=0.9339867511378304,50.14520994379217,6.207424251137831,51.77292299745045&overlay=false&bgLayer=MapBox&date=2019-02-25T03:17:15.000Z&layer=CGS_S2_FAPAR',
    'rel': 'alternate',
    'title': 'Terrascope Viewer'},
   {'href': 'https://services.terrascope.be/wmts/v2',
    'rel': 'wmts',
    'wmts:layer': 'CGS_S2_FAPAR'}],
  'providers': [{'name': 'VITO, on

Let's take Sentinel-2 radiometry data.

In [5]:
collection_id = "TERRASCOPE_S2_NDVI_V2"
session.describe_collection(collection_id)

{'assets': {'metadata_iso_19139': {'href': 'https://services.terrascope.be/collectioncatalogue/srv/api/records/urn:eop:VITO:TERRASCOPE_S2_NDVI_V2/formatters/xml?approved=true',
   'roles': ['metadata', 'iso-19139'],
   'title': 'ISO 19139 metadata',
   'type': 'application/vnd.iso.19139+xml'},
  'thumbnail': {'href': 'https://services.terrascope.be/collectioncatalogue/images/quicklooks/SENTINEL2_NDVI_V2_QL.JPG',
   'roles': ['thumbnail'],
   'title': 'Thumbnail',
   'type': 'image/jpeg'}},
 'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['NDVI_10M', 'SCENECLASSIFICATION_20M']},
  't': {'extent': ['2015-07-06T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x', 'extent': [-180, 180], 'step': 10, 'type': 'spatial'},
  'y': {'axis': 'y', 'extent': [-56, 83], 'step': 10, 'type': 'spatial'}},
 'description': 'NDVI Level-3 product at 10m resolution. Covers selected areas globally. This NDVI is derived directly from ESA L2A products.',
 'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
  'temporal': {'interval': [['2015-07-06T00:00:00Z', None]]}},
 'id': 'TERRASCOPE_S2_NDVI_V2',
 'keywords': ['VITO',
  'TERRASCOPE',
  'COPERNICUS',
  'ESA',
  'Orthoimagery',
  'SENTINEL-2',
  'MSI',
  'Level-3',
  'Radiometry',
  'Plant Resource',
  'NDVI'],
 'license': 'proprietary',
 'links': [{'href': 'https://sentinel.esa.int/documents/247904/690755/Sentinel_Data_Legal_Notice',
   'rel': 'license'},
  {'href': 'https://services.terrascope.be/catalogue/description?collection=urn:eop:VITO:TERRASCOPE_S2_NDVI_V2',
   'rel': 'alternate'},
  {'href': 'https://viewer.terrascope.be/?language=nl&bbox=0.9339867511378304,50.14520994379217,6.207424251137831,51.77292299745045&overlay=false&bgLayer=MapBox&date=2019-02-25T03:17:15.000Z&layer=CGS_S2_NDVI',
   'rel': 'alternate'},
  {'href': 'https://services.terrascope.be/wmts/v2',
   'rel': 'wmts',
   'wmts:layer': 'CGS_S2_NDVI'}],
 'providers': [{'name': 'VITO, on behalf of the Belgian Science Policy Office (BELSPO).',
   'roles': ['producer', 'licensor'],
   'url': 'https://terrascope.be'}],
 'stac_extensions': ['datacube'],
 'stac_version': '0.9.0',
 'summaries': {'eo:bands': [{'name': 'NDVI_10M'},
   {'name': 'SCENECLASSIFICATION_20M'}],
  'instruments': ['MSI'],
  'platform': ['SENTINEL-2']},
 'title': 'Sentinel-2 NDVI last 2 years over Europe + selected areas, by Terrascope.'}

In the client code we create a so called image collection, which is a thin reference to this data and allows to build on that with further operations.

In [6]:
s2_datacube = session.load_collection(collection_id).band('NDVI_10M')
s2_datacube

## Creating composite

Next we start by creating our composite. As the data covered by the image collection can be quite large, a first step is usually to define a spatial and temporal subset on which we want to operate. This can be done by specifying a date range and a bounding box:

In [7]:
timeseries = s2_datacube\
.filter_temporal("2020-07-01","2020-07-31")\
.filter_bbox(west=2.88254,east=3.24234,south=51.13243,north=51.45653,crs="EPSG:4326")
timeseries

Now we need to specify how OpenEO needs to combine the data to create a composite. This can be done by applying one of the available functions such as `max_time`, `mean_time`, `min_time`. A full overview of all operations that can be applied to a datacube is available in the [OpenEO documentation](https://open-eo.github.io/openeo-python-client/api.html?highlight=max_time#openeo.rest.datacube.DataCube).

For example, the `max_time` function allows us to specify the function that needs to be applied, but does not yet compute a result :

In [8]:
composite = timeseries.max_time()
composite

The last step is to store the results, execute the process as a batch job and download the results. The composite will be downloaded locally to a filed called `composite.tif`.

In [9]:
composite_job = composite.save_result(format='gtiff').send_job()
composite_job.start_and_wait().get_results().download_file("./composite.tiff")

0:00:00 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': send 'start'
0:00:57 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:01:02 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:01:09 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:01:17 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:01:27 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:01:40 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:01:56 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:02:15 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:02:39 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': queued (progress N/A)
0:03:10 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': running (progress N/A)
0:03:47 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': running (progress N/A)
0:04:35 Job '76950a6f-045c-4472-9fca-01a8bf3fd0cb': running (progress N/A)
0:05:33 Job '76950a6f-045c-4472-9fca-01a8bf3

PosixPath('composite.tiff')